In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
file_dir = '/content/drive/MyDrive/데이콘/도배 하자 유형 분류'

In [3]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 


In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [5]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}


In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정


In [20]:
all_img_list = glob.glob(file_dir + '/train/*/*')

In [38]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [39]:
df

,img_path,label
0,/content/drive/MyDrive/데이콘/도배 하자 유형...,걸레받이수정
1,/content/drive/MyDrive/데이콘/도배 하자 유형...,걸레받이수정
2,/content/drive/MyDrive/데이콘/도배 하자 유형...,걸레받이수정
3,/content/drive/MyDrive/데이콘/도배 하자 유형...,걸레받이수정
4,/content/drive/MyDrive/데이콘/도배 하자 유형...,걸레받이수정
...,...,...
3452,/content/drive/MyDrive/데이콘/도배 하자 유형...,훼손
3453,/content/drive/MyDrive/데이콘/도배 하자 유형...,훼손
3454,/content/drive/MyDrive/데이콘/도배 하자 유형...,훼손
3455,/content/drive/MyDrive/데이콘/도배 하자 유형...,훼손


In [40]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [41]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [42]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)


In [43]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])


In [44]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


In [45]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


In [46]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model


In [47]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score


In [48]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 63.0MB/s]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.40529] Val Loss : [0.91723] Val Weighted F1 Score : [0.69424]


  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.05118] Val Loss : [1.13600] Val Weighted F1 Score : [0.76967]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.01581] Val Loss : [1.15827] Val Weighted F1 Score : [0.76764]


In [60]:
test = pd.read_csv(file_dir + '/test.csv')

In [61]:
test['img_path'] = test['img_path'].str.replace('./test', file_dir + '/test')
test

,id,img_path
0,TEST_000,/content/drive/MyDrive/데이콘/도배 하자 유형...
1,TEST_001,/content/drive/MyDrive/데이콘/도배 하자 유형...
2,TEST_002,/content/drive/MyDrive/데이콘/도배 하자 유형...
3,TEST_003,/content/drive/MyDrive/데이콘/도배 하자 유형...
4,TEST_004,/content/drive/MyDrive/데이콘/도배 하자 유형...
...,...,...
787,TEST_787,/content/drive/MyDrive/데이콘/도배 하자 유형...
788,TEST_788,/content/drive/MyDrive/데이콘/도배 하자 유형...
789,TEST_789,/content/drive/MyDrive/데이콘/도배 하자 유형...
790,TEST_790,/content/drive/MyDrive/데이콘/도배 하자 유형...


In [62]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [96]:
[(i,label) for i,label in enumerate(le.classes_)]

[(0, '가구수정'),
 (1, '걸레받이수정'),
 (2, '곰팡이'),
 (3, '꼬임'),
 (4, '녹오염'),
 (5, '들뜸'),
 (6, '면불량'),
 (7, '몰딩수정'),
 (8, '반점'),
 (9, '석고수정'),
 (10, '오염'),
 (11, '오타공'),
 (12, '울음'),
 (13, '이음부불량'),
 (14, '창틀,문틀수정'),
 (15, '터짐'),
 (16, '틈새과다'),
 (17, '피스'),
 (18, '훼손')]

In [84]:
def label_decoder(pred):
	if pred==0:
		return '가구수정'
	if pred==1:
		return '걸레받이수정'
	if pred==2:	
		return '곰팡이' 
	if pred==3:	
		return '꼬임'   
	if pred==4:	
		return '녹오염' 
	if pred==5:	
		return '들뜸' 
	if pred==6:	
		return '면불량' 
	if pred==7:	
		return '몰딩수정'
	if pred==8:	
		return '반점' 
	if pred==9:	
		return '석고수정'
	if pred==10:	
		return '오염' 
	if pred==11:	
		return '오타공' 
	if pred==12:	
		return '울음' 
	if pred==13:	
		return '이음부불량' 
	if pred==14:	
		return '창틀,문틀수정' 
	if pred==15:	
		return '터짐' 
	if pred==16:	
		return '틈새과다'
	if pred==17:	
		return '피스' 
	if pred==18:	
		return '훼손' 
	else:
		return 'NaN' 

In [88]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()            
    
    preds = [label_decoder(i) for i in preds]
    return preds


In [89]:
preds = inference(infer_model, test_loader, device)


  0%|          | 0/25 [00:00<?, ?it/s]

In [91]:
submit = pd.read_csv(file_dir+'/sample_submission.csv')

In [92]:
submit['label'] = preds

In [93]:
submit.loc[submit['label'] == '0', 'label'] = '가구수정'
submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
submit.loc[submit['label'] == '3', 'label'] = '꼬임'
submit.loc[submit['label'] == '4', 'label'] = '녹오염'
submit.loc[submit['label'] == '5', 'label'] = '들뜸'
submit.loc[submit['label'] == '6', 'label'] = '면불량'
submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
submit.loc[submit['label'] == '8', 'label'] = '반점'
submit.loc[submit['label'] == '9', 'label'] = '석고수정'
submit.loc[submit['label'] == '10', 'label'] = '오염'
submit.loc[submit['label'] == '11', 'label'] = '오타공'
submit.loc[submit['label'] == '12', 'label'] = '울음'
submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == '15', 'label'] = '터짐'
submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
submit.loc[submit['label'] == '17', 'label'] = '피스'
submit.loc[submit['label'] == '18', 'label'] = '훼손'

In [95]:
submit.to_csv(file_dir+'/baseline_submit.csv', index=False)